<a href="https://colab.research.google.com/github/JuanHermann/BT_IFR2/blob/main/BT_IFR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from tabulate import tabulate

In [53]:
def rsi2_mm3(data, rsi_period=2, smooth_period=3):

    if "Close" not in data.columns or len(data) < smooth_period:
        print(f"Erro: Dados insuficientes. Necessário pelo menos {smooth_period} períodos.")
        return data

    # 2. Variação de Preço
    delta = data["Close"].diff(periods=1)

    # 3. Ganhos (Up) e Perdas (Down) - Perda como valor positivo
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # 4. Suavização EMA (Wilder's Smoothing)
    # O método 'ewm' com adjust=False e span=smooth_period é o equivalente no Pandas
    # à suavização de Welles Wilder (RMA).

    # Usamos min_periods=smooth_period para o cálculo começar no ponto correto.
    avg_gain = gain.ewm(span=smooth_period, min_periods=smooth_period, adjust=False).mean()
    avg_loss = loss.ewm(span=smooth_period, min_periods=smooth_period, adjust=False).mean()

    # 5. Cálculo do RS e RSI
    epsilon = 1e-10 # Adiciona epsilon para estabilidade numérica e evitar divisão por zero

    # Força Relativa (RS)
    rs = avg_gain / (avg_loss + epsilon)

    # Índice de Força Relativa (RSI)
    rsi = 100 - (100 / (1 + rs))

    # 6. Tratamento de Casos Extremos (Divisão por zero)
    # Garante que o RSI = 100 quando não há perda (avg_loss = 0) e há ganho.
    loss_is_zero = np.isclose(avg_loss, 0, atol=epsilon)
    gain_is_positive = (avg_gain > 0)

    rsi[loss_is_zero & gain_is_positive] = 100

    # 7. Renomeia e Retorna
    # Nome da coluna ajustado para refletir o período de suavização real
    data["RSI_2"] = rsi

    # Retorna o DataFrame a partir do ponto onde o RSI é calculado
    return data.iloc[smooth_period - 1:]

In [54]:
def exitTrade(trades,entry_date, exit_date, entry_price,exit_price,max_holding_days):

  profit = exit_price - entry_price
  trades.append({
                'Entry_Date': entry_date,
                'Exit_Date': exit_date,
                'Entry_Price': entry_price,
                'Exit_Price': exit_price,
                'P/L': profit,
                'Result': 'Win' if profit > 0 else 'Loss',
                'Max_Hold_Days': max_holding_days

            })

In [55]:
def gerar_tabela(list_of_trades_data):

    cabecalho_multi_nivel = []
    periodos = ["5 ANOS", "3 ANOS", "1 ANO", "6 MESES"]
    metricas = ["QTDE", "F. LUCRO", "%"]

    for periodo in periodos:
        for metrica in metricas:
            cabecalho_multi_nivel.append((periodo, metrica))

    cabecalho_multi_nivel.append(("CANDLES", ""))
    cabecalho_multi_nivel.append(("DROPDOWN MAX", ""))

    colunas = pd.MultiIndex.from_tuples(cabecalho_multi_nivel, names=['Período', 'Métrica'])

    # --- 2. Processar cada conjunto de trades na lista ---
    resultados = []
    for trades_data in list_of_trades_data:
        # Convert the trades data to a DataFrame if it's not already
        trades_df = pd.DataFrame(trades_data)

        if trades_df.empty:
            # Append a row of zeros or NaNs if there are no trades for this set
            resultados.append([0] * len(colunas))
            continue

        # Ensure 'Entry_Date' is datetime for filtering
        trades_df['Entry_Date'] = pd.to_datetime(trades_df['Entry_Date'])

        # Filter trades based on time periods
        trades_df_5_years  = trades_df[trades_df['Entry_Date'] >= (date.today() - pd.DateOffset(years=5))]
        trades_df_3_years  = trades_df[trades_df['Entry_Date'] >= (date.today() - pd.DateOffset(years=3))]
        trades_df_1_year   = trades_df[trades_df['Entry_Date'] >= (date.today() - pd.DateOffset(years=1))]
        trades_df_6_months = trades_df[trades_df['Entry_Date'] >= (date.today() - pd.DateOffset(months=6))]

        # Calculate metrics for each period
        row_data = [
            # 5 ANOS (QTDE, F. LUCRO, %)
            len(trades_df_5_years), getProfitFactor(trades_df_5_years), getWinRate(trades_df_5_years),
            # 3 ANOS (QTDE, F. LUCRO, %)
            len(trades_df_3_years), getProfitFactor(trades_df_3_years), getWinRate(trades_df_3_years),
            # 1 ANO (QTDE, F. LUCRO, %)
            len(trades_df_1_year), getProfitFactor(trades_df_1_year), getWinRate(trades_df_1_year),
            # 6 MESES (QTDE, F. LUCRO, %)
            len(trades_df_6_months), getProfitFactor(trades_df_6_months), getWinRate(trades_df_6_months),
            # CANDLES (Assuming you want an aggregate like mean)
            trades_df['Max_Hold_Days'][0],
            # DROP MAX
            getMaxDropdown(trades_df)
        ]
        resultados.append(row_data)


    # --- 3. CRIAR O DATAFRAME ---
    df = pd.DataFrame(resultados, columns=colunas)
    df.index.name = None # Opcional: Remover o nome do índice da primeira linha

    return df
#gerar_tabela(trades_list_of_lists)

In [56]:
def getWinRate(trades_df):
    winning_trades = len(trades_df[trades_df['P/L'] > 0])
    win_rate = (winning_trades / len(trades_df)) * 100
    return f"{win_rate:.2f}%"

In [57]:
def getProfitFactor(trades_df):
    total_gross_profit = trades_df[trades_df['P/L'] > 0]['P/L'].sum()
    total_gross_loss = abs(trades_df[trades_df['P/L'] < 0]['P/L'].sum())
    profit_factor = total_gross_profit / total_gross_loss if total_gross_loss > 0 else np.inf
    return f"{profit_factor:.2f}"

In [58]:
def getMaxDropdown(trades_df):
    # 6.3. Drawdown Máximo (Max Drawdown)
    # Calcula o retorno cumulativo (equity curve)
    trades_df['Cumulative_P/L'] = trades_df['P/L'].cumsum()
    equity_curve = trades_df['Cumulative_P/L']
    # Calcula o máximo valor da curva de patrimônio até o momento
    peak = equity_curve.expanding().max()
    # Calcula o Drawdown (do pico até o valor atual)
    drawdown = (equity_curve - peak) / peak.mask(peak == 0, 1)
    max_drawdown = abs(drawdown.min()) * 100
    return f"{max_drawdown:.2f}%"

In [59]:
def getTableAllResults(all_tickers_results):

  # Create an empty list to store the data for the new DataFrame
  data_for_new_df = []

  # Iterate through the results for each ticker
  for ticker, result_df in all_tickers_results.items():
    # Check if the result is a DataFrame and not empty
    if isinstance(result_df, pd.DataFrame) and not result_df.empty:
      # Get the first row of the results DataFrame
      best_row = selecionar_melhor_parametro_ponderado(result_df)
      # Add the ticker to the dictionary for the first row
      best_row['Ticker'] = ticker[:-3]
      # Append the data to the list
      data_for_new_df.append(best_row)
    else:
      # Handle cases where there are no results or an error occurred
      print(f"Could not get results for {ticker}: {result_df}")


  # Create a new DataFrame from the collected data
  new_df = pd.DataFrame(data_for_new_df)

  # Reorder columns to have Ticker as the first column
  if 'Ticker' in new_df.columns:
    cols = ['Ticker'] + [col for col in new_df.columns if col != 'Ticker']
    new_df = new_df[cols]

  return new_df

In [60]:
def selecionar_melhor_parametro_ponderado(df_resultados: pd.DataFrame) -> pd.DataFrame:

    # 1. Preparação: Limpeza, normalização e cálculo do Score para CADA PERÍODO

    # Ensure index name is set for the score DataFrame
    df_scores = df_resultados.index.to_frame(name='CANDLES').copy()


    for periodo, peso in PERIODOS_E_PESOS.items():

        # Extrai as métricas
        qtde = df_resultados[(periodo, 'QTDE')]

        # Convert 'F. LUCRO' to numeric, coercing errors
        f_lucro = pd.to_numeric(df_resultados[(periodo, 'F. LUCRO')], errors='coerce')

        # Convert '%' to numeric (float between 0 and 1), coercing errors
        taxa_acerto_str = df_resultados[(periodo, '%')].astype(str).str.replace('%', '').str.replace(',', '.')
        taxa_acerto = pd.to_numeric(taxa_acerto_str, errors='coerce') / 100

        # Create a column for the Raw Score of this period
        coluna_score_bruto = f'SCORE_BRUTO_{periodo.replace(" ", "_")}'

        # Formula for Raw Score (Quality)
        # Score = (Win Rate * Profit Factor) * sqrt(Quantity of Trades)
        # Handle potential NaN values from coercion by filling with 0 for score calculation
        df_scores[coluna_score_bruto] = (taxa_acerto.fillna(0) * f_lucro.fillna(0) * np.sqrt(qtde.fillna(0)))

        # Apply the Weight to the Final Score
        df_scores[coluna_score_bruto] *= peso

    # 2. Calculation of FINAL SCORE (Weighted Sum)

    score_cols = [col for col in df_scores.columns if col.startswith('SCORE_BRUTO')]
    df_scores['SCORE_FINAL_PONDERADO'] = df_scores[score_cols].sum(axis=1)

    # 3. Selection of the Best Result

    melhor_parametro_index = df_scores['SCORE_FINAL_PONDERADO'].idxmax()

    # Return the final result, including the complete row from the original DF and the Final Score
    melhor_linha = df_resultados.loc[melhor_parametro_index].to_frame().T
    # Add the SCORE FINAL PONDERADO to the output DataFrame
    melhor_linha[('SCORE', 'FINAL')] = df_scores.loc[melhor_parametro_index, 'SCORE_FINAL_PONDERADO']

    return  df_resultados.iloc[melhor_parametro_index].to_dict()

In [61]:
def showAllResults(all_tickers_results):
# --- Display results for all tickers ---
  print("\n--- Backtest Results for All Tickers ---")
  for ticker, result in all_tickers_results.items():
      print(f"\nResults for {ticker}:")
      if isinstance(result, pd.DataFrame):
          display(result)
      else:
          print(result)

In [65]:
# -------------ANOTACOES -----------
# A QUANTIDADE DE TRADES PODE SEM 1 NUMERO MENOR QUE O PROFIT POIS O PROFIT CONTA OS TRADES NAO FINALIZADOS.

# -------------- TO DO -------------
# CONFERIR O CALCULO DO MAX DROPDOWN
# VALIDAR ALGORITIMO QUE DECIDE A MELHOR LINHA
# CONFERIR ODPV3 COM F. LUCRO 'INF' SENDO QUE TEVE 16 TRADES



TICKERS = ["CPLE6.SA","CPFE3.SA","MOTV3.SA","COGN3.SA","IRBR3.SA","ELET6.SA",
           "VIVT3.SA","MULT3.SA","SBSP3.SA","EQTL3.SA","ITUB4.SA","ODPV3.SA",
           "GRND3.SA","EGIE3.SA","TOTS3.SA","TIMS3.SA","ABEV3.SA","PRIO3.SA",
           "PETR4.SA","RAIL3.SA","PSSA3.SA","WEGE3.SA","GGBR4.SA","ENEV3.SA",
           "ISAE4.SA","TAEE11.SA","CMIG4.SA","EZTC3.SA","HYPE3.SA","TGMA3.SA",
           "RENT3.SA","SANB11.SA","BOVA11.SA","SLCE3.SA","EMBR3.SA","SUZB3.SA"]
PERIODOS_E_PESOS = {
        "5 ANOS": 1.0,
        "3 ANOS": 0.8,
        "1 ANO": 0.6,
        "6 MESES": 0.4,
    }
RSI_PERIOD = 2
RSI_THRESHOLD = 25
MAX_HOLDING_DAYS_RANGE = range(3, 9)
START_DATE = date.today() - pd.DateOffset(years=5)
DIAS_ANTERIORES = 30

all_tickers_results = {}

start_date_download = START_DATE - pd.DateOffset(days=DIAS_ANTERIORES)
print(f"realizando Backtest entre as datas {START_DATE.strftime("%d/%m/%Y")} e {date.today().strftime("%Y-%m-%d")}")

# --- 3. Obter Dados para todos os Tickers ---
try:
    print(f"Baixando dados para {TICKERS} desde {start_date_download}...")
    data = yf.download(TICKERS, start=start_date_download, end=date.today().strftime("%Y-%m-%d"), auto_adjust=True)

    if data.empty:
        raise ValueError(f"Não foi possível baixar dados para os tickers {TICKERS}. Verifique os símbolos.")

except Exception as e:
    print(f"ERRO: Não foi possível obter dados para os tickers {TICKERS}. Verifique sua conexão ou os tickers. Detalhe: {e}")



# --- Process data and perform backtest for each ticker ---
# Iterate through each ticker to process its data and run the backtest
for ticker in TICKERS:
    print(f"--- Processing Ticker: {ticker} ---")

    # Select the data for the current ticker from the multi-indexed DataFrame
    # Check if data was successfully downloaded and the ticker exists in the downloaded data
    if 'data' in locals() and not data.empty and ticker in data.columns.get_level_values(1):
        # Select all rows (:) and columns where the second level of the MultiIndex (ticker) matches the current ticker
        df = data.xs(ticker, level='Ticker', axis=1).copy()

        df = rsi2_mm3(df)

        # --- 4. Pré-processamento dos Dados ---
        # Remove linhas com valores nulos (início dos cálculos)
        df.dropna(inplace=True)

        trades_list_of_lists = [] # This will store trades for each maxHoldDay run for the current ticker

        # faz todos os testes para verificar qual a melhor quantidade de dias para levar a operacao
        for maxHoldDay in MAX_HOLDING_DAYS_RANGE:
            trades = [] # List to store trades for the current maxHoldDay
            in_trade = False
            entry_price = 0
            entry_date = None
            exit_target = 0
            holding_days = 0

            for i in range(len(df)):
                current_day = df.iloc[i]

                if current_day.name < START_DATE:
                    continue

                if not in_trade:
                    # Condição de Compra (Entrada): RSI(2) abaixo de 25

                    if current_day['RSI_2'].item() < RSI_THRESHOLD:
                        in_trade = True
                        entry_price = current_day['Close'].item()
                        entry_date = current_day.name
                        holding_days = 0


                else: # Está em um trade
                    holding_days += 1

                    exit_target= max(df.iloc[i-1]['High'].item(),df.iloc[i-2]['High'].item())

                    # 1. Condição de Venda (Take Profit): Preço atinge ou supera o alvo
                    if current_day['High'].item() >= exit_target:
                        exitTrade(trades,entry_date,current_day.name,entry_price,exit_target, maxHoldDay)
                        in_trade = False
                        continue

                    # 2. Condição de Venda (Time Stop): Fechamento do candle limite
                    if holding_days >= maxHoldDay:
                        exitTrade(trades,entry_date,current_day.name,entry_price,current_day['Close'].item(), maxHoldDay)
                        in_trade = False
                        continue


            # Case where the last trade was not closed for this maxHoldDay run
            if in_trade:
                #print(f"Attention: Last trade not finished for MAX_HOLDING_DAYS = {maxHoldDay} for ticker {ticker}. Ignored for metrics.")
                pass # Or handle as needed, e.g., close at the last price if desired


            # Append the trades list for the current maxHoldDay to the list of lists
            if trades:
                trades_list_of_lists.append(trades)


        # --- 6. Cálculo das Métricas de Desempenho e Geração da Tabela ---
        if trades_list_of_lists:
            # Pass the list of trades data to the gerar_tabela function and store in the dictionary
            all_tickers_results[ticker] = gerar_tabela(trades_list_of_lists)
        else:
            print(f"\nNenhum trade realizado para o ticker {ticker} no período de backtest com as condições especificadas para nenhum MAX_HOLDING_DAYS.")
            all_tickers_results[ticker] = "No trades found"
    else:
        # If data download failed for this specific ticker or ticker not found in downloaded data, skip processing
        if ticker not in all_tickers_results: # Avoid overwriting existing error message if download failed
             all_tickers_results[ticker] = "Ticker not found in downloaded data or data download failed"
        print(f"Skipping processing for {ticker} due to data issue.")

#showAllResults(all_tickers_results)
sorted_df = getTableAllResults(all_tickers_results)
sorted_df.sort_values(by=('3 ANOS', 'F. LUCRO'), ascending=False)
display(sorted_df)

realizando Backtest entre as datas 10/11/2020 e 2025-11-10
Baixando dados para ['CPLE6.SA', 'CPFE3.SA', 'MOTV3.SA', 'COGN3.SA', 'IRBR3.SA', 'ELET6.SA', 'VIVT3.SA', 'MULT3.SA', 'SBSP3.SA', 'EQTL3.SA', 'ITUB4.SA', 'ODPV3.SA', 'GRND3.SA', 'EGIE3.SA', 'TOTS3.SA', 'TIMS3.SA', 'ABEV3.SA', 'PRIO3.SA', 'PETR4.SA', 'RAIL3.SA', 'PSSA3.SA', 'WEGE3.SA', 'GGBR4.SA', 'ENEV3.SA', 'ISAE4.SA', 'TAEE11.SA', 'CMIG4.SA', 'EZTC3.SA', 'HYPE3.SA', 'TGMA3.SA', 'RENT3.SA', 'SANB11.SA', 'BOVA11.SA', 'SLCE3.SA', 'EMBR3.SA', 'SUZB3.SA'] desde 2020-10-11 00:00:00...


[*********************100%***********************]  36 of 36 completed


--- Processing Ticker: CPLE6.SA ---
--- Processing Ticker: CPFE3.SA ---
--- Processing Ticker: MOTV3.SA ---
--- Processing Ticker: COGN3.SA ---
--- Processing Ticker: IRBR3.SA ---
--- Processing Ticker: ELET6.SA ---
--- Processing Ticker: VIVT3.SA ---
--- Processing Ticker: MULT3.SA ---
--- Processing Ticker: SBSP3.SA ---
--- Processing Ticker: EQTL3.SA ---
--- Processing Ticker: ITUB4.SA ---
--- Processing Ticker: ODPV3.SA ---
--- Processing Ticker: GRND3.SA ---
--- Processing Ticker: EGIE3.SA ---
--- Processing Ticker: TOTS3.SA ---
--- Processing Ticker: TIMS3.SA ---
--- Processing Ticker: ABEV3.SA ---
--- Processing Ticker: PRIO3.SA ---
--- Processing Ticker: PETR4.SA ---
--- Processing Ticker: RAIL3.SA ---
--- Processing Ticker: PSSA3.SA ---
--- Processing Ticker: WEGE3.SA ---
--- Processing Ticker: GGBR4.SA ---
--- Processing Ticker: ENEV3.SA ---
--- Processing Ticker: ISAE4.SA ---
--- Processing Ticker: TAEE11.SA ---
--- Processing Ticker: CMIG4.SA ---
--- Processing Ticker: EZTC

,Ticker,"(5 ANOS, QTDE)","(5 ANOS, F. LUCRO)","(5 ANOS, %)","(3 ANOS, QTDE)","(3 ANOS, F. LUCRO)","(3 ANOS, %)","(1 ANO, QTDE)","(1 ANO, F. LUCRO)","(1 ANO, %)","(6 MESES, QTDE)","(6 MESES, F. LUCRO)","(6 MESES, %)","(CANDLES, )","(DROPDOWN MAX, )"
0,CPLE6,131,2.72,77.10%,82,4.23,78.05%,24,6.76,79.17%,14,3.93,78.57%,7,134.14%
1,CPFE3,135,2.09,73.33%,81,2.37,77.78%,27,4.32,81.48%,14,2.76,78.57%,7,169.02%
2,MOTV3,14,4.82,85.71%,14,4.82,85.71%,14,4.82,85.71%,12,4.11,83.33%,6,28.03%
3,COGN3,131,0.98,63.36%,78,1.27,67.95%,20,3.46,80.00%,13,3.29,84.62%,7,335.48%
4,IRBR3,134,0.86,58.96%,75,1.31,57.33%,25,3.44,72.00%,12,2.44,66.67%,6,306.99%
5,ELET6,127,1.50,69.29%,78,1.75,70.51%,26,6.12,84.62%,13,2.94,69.23%,7,263.38%
6,VIVT3,125,2.01,77.60%,72,2.41,77.78%,21,2.60,76.19%,12,1.65,66.67%,7,138.90%
7,MULT3,170,1.68,68.24%,99,2.00,70.71%,30,1.57,73.33%,17,2.49,82.35%,3,122.58%
8,SBSP3,141,1.55,65.96%,82,1.91,69.51%,23,2.33,69.57%,12,1.45,66.67%,4,142.57%
9,EQTL3,163,1.63,63.19%,102,1.69,61.76%,28,1.66,75.00%,15,1.97,73.33%,3,289.17%


In [66]:
# TABELA DOS RESULTADOS DOS CANDLES
#display(all_tickers_results['GRND3.SA'])

#data.xs('MOTV3.SA', level='Ticker', axis=1)
new =getTableAllResults(all_tickers_results)
new_sorted = new.sort_values(by=('5 ANOS', 'F. LUCRO'), ascending=False)
display(new_sorted)

,Ticker,"(5 ANOS, QTDE)","(5 ANOS, F. LUCRO)","(5 ANOS, %)","(3 ANOS, QTDE)","(3 ANOS, F. LUCRO)","(3 ANOS, %)","(1 ANO, QTDE)","(1 ANO, F. LUCRO)","(1 ANO, %)","(6 MESES, QTDE)","(6 MESES, F. LUCRO)","(6 MESES, %)","(CANDLES, )","(DROPDOWN MAX, )"
2,MOTV3,14,4.82,85.71%,14,4.82,85.71%,14,4.82,85.71%,12,4.11,83.33%,6,28.03%
0,CPLE6,131,2.72,77.10%,82,4.23,78.05%,24,6.76,79.17%,14,3.93,78.57%,7,134.14%
1,CPFE3,135,2.09,73.33%,81,2.37,77.78%,27,4.32,81.48%,14,2.76,78.57%,7,169.02%
6,VIVT3,125,2.01,77.60%,72,2.41,77.78%,21,2.60,76.19%,12,1.65,66.67%,7,138.90%
11,ODPV3,134,1.87,67.16%,80,1.95,67.50%,29,3.84,82.76%,15,inf,100.00%,7,78.46%
26,CMIG4,139,1.79,72.66%,85,2.45,77.65%,33,6.04,81.82%,16,5.55,81.25%,4,198.99%
7,MULT3,170,1.68,68.24%,99,2.00,70.71%,30,1.57,73.33%,17,2.49,82.35%,3,122.58%
24,ISAE4,134,1.68,70.90%,77,1.52,71.43%,22,1.61,77.27%,11,2.05,72.73%,6,178.72%
9,EQTL3,163,1.63,63.19%,102,1.69,61.76%,28,1.66,75.00%,15,1.97,73.33%,3,289.17%
10,ITUB4,140,1.59,71.43%,82,2.13,75.61%,25,1.93,76.00%,13,1.39,76.92%,6,86.64%
